In [ ]:
#  Load the 102 Flowers dataset using TensorFlow
import tensorflow_datasets as tfds
flowers102, info = tfds.load('oxford_flowers102', split='train', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.FEJKV2_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.FEJKV2_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-012sw976
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-012sw976
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [ ]:
import os
import clip
import torch

In [ ]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 109MiB/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

#for image, label in flowers102.take(1):
 #   print(f"Image shape: {image.shape}")
  #  print(f"Label: {label}")

In [ ]:
# 2. Prepare the inputs
from PIL import Image
image, class_id = next(iter(flowers102.take(1)))
image = Image.fromarray(image.numpy().astype('uint8'), 'RGB')
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in info.features['label'].names]).to(device)

In [ ]:
#  features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

In [ ]:
#  top 5 most similar labels for  image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

In [ ]:
# 4. Print the top predictions
print("\nTop predictions:\n")
for value, index in zip(values.squeeze(), indices.squeeze()):
    print(f"{info.features['label'].names[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

      water lily: 57.31%
           lotus: 27.27%
      sword lily: 6.35%
      siam tulip: 2.26%
        bromelia: 1.40%


In [ ]:
import os
import clip
import torch
from torchvision.datasets import Flowers102
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision.transforms import ToTensor

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [ ]:
# Load the dataset
root = os.path.expanduser("~/.cache")
# train test split
train_dataset = Flowers102(root, split="train", transform=preprocess, download=True)
test_dataset = Flowers102(root, split="test", transform=preprocess, download=True)

In [ ]:
def get_features(dataset):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [ ]:
# Calculate the image features
train_features, train_labels = get_features(train_dataset)
test_features, test_labels = get_features(test_dataset)


100%|██████████| 62/62 [19:05<00:00, 18.47s/it]


In [ ]:
# logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

LogisticRegression(C=0.316, max_iter=1000, random_state=0, verbose=1)

In [ ]:
#  logistic regression classifier
predictions = classifier.predict(test_features)
#accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
#print(f"Accuracy = {accuracy:.3f}")

In [ ]:
#  unique class
class_labels = np.unique(info.features['label'].names)

In [ ]:
# store class-wise
class_accuracy = {label: 0 for label in class_labels}
class_counts = {label: 0 for label in class_labels}

In [ ]:
# Calculate class-wise
for true_label, pred_label in zip(test_labels, predictions):
    class_counts[info.features['label'].names[true_label]] += 1
    if true_label == pred_label:
        class_accuracy[info.features['label'].names[true_label]] += 1

In [ ]:
# Calculate accuracy
overall_accuracy = np.mean((test_labels == predictions).astype(float)) * 100.

In [ ]:
#  overall accuracy
print(f"Overall Accuracy = {overall_accuracy:.3f}")

Overall Accuracy = 93.283


**the best accuracy belong to these** **classes**

In [ ]:
# Print class-wise accuracy
print("\nClass-wise Accuracy:")
for label in class_labels:
    accuracy = class_accuracy[label] / class_counts[label] * 100
    print(f"{label:>16s}: {accuracy:.3f}%")


Class-wise Accuracy:
alpine sea holly: 100.000%
       anthurium: 95.294%
       artichoke: 98.276%
          azalea: 96.053%
       ball moss: 88.462%
  balloon flower: 89.655%
  barbeton daisy: 96.262%
    bearded iris: 100.000%
        bee balm: 91.304%
bird of paradise: 98.462%
bishop of llandaff: 100.000%
black-eyed susan: 100.000%
 blackberry lily: 100.000%
  blanket flower: 100.000%
bolero deep blue: 85.000%
   bougainvillea: 92.593%
        bromelia: 95.349%
       buttercup: 92.157%
californian poppy: 98.780%
        camellia: 76.056%
      canna lily: 75.806%
canterbury bells: 40.000%
     cape flower: 97.727%
       carnation: 100.000%
cautleya spicata: 100.000%
        clematis: 90.217%
     colt's foot: 92.537%
       columbine: 86.364%
common dandelion: 93.056%
      corn poppy: 95.238%
        cyclamen: 92.537%
        daffodil: 97.436%
     desert-rose: 93.023%
english marigold: 93.333%
       fire lily: 100.000%
        foxglove: 93.662%
      frangipani: 99.315%
    

***5 top class that have the lowest accuracy:***
